In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

In [ ]:
inpath = "iHil3966.xls"
reac = pd.read_excel(inpath)
df2 = pd.read_excel(inpath,sheet_name="Metabolite List")
df3 = df2.iloc[:,0:2]
reaction = list(reac["Reaction"])
all_left, all_right = exo_metabolic_1(reaction)
    
df = exo_metabolic_2(all_left,all_right)
df4 = pd.merge(df,df3,on="Abbreviation")

In [ ]:

met_all=list(df2['Abbreviation'])
met_all2=[]
for i in met_all:
    if i[-3:] == '[e]':
        met_all2.append(i)
met_all3=[x[:-4] for x in met_all2]

In [ ]:
all_left2=[x[:-4] for x in all_left]
all_right2=[x[:-4] for x in all_right]

In [ ]:
df_new = exo_metabolic_2(all_left2,all_right2)
new_met=list(df3['Abbreviation'])
new_met2=[x[:-4] for x in new_met]
df3_new=pd.DataFrame({'Abbreviation':new_met2,'Description':list(df3['Description'])})
df4_new = pd.merge(df_new,df3_new,on="Abbreviation")
df4_new_unique = df4_new.drop_duplicates(subset=['Abbreviation'],keep='first')

In [ ]:
set1 = set(list(df4_new_unique['Abbreviation']))
set3 = set(met_all3)

In [ ]:
exchange=list(set1 & set3)
rxn_index1=[]
for i in reaction_list:
    if '<=>' in i: 
        xx=i.split('  <=> ')[0]
        rxn_index1.append(xx)
    if '->' in i:
        xx=i.split('  -> ')[0]
        rxn_index1.append(xx)
        

In [ ]:
rxn_index2 = []
for i in exchange:
    xx2 = i + 'e[e]'
    if xx2 in rxn_index1:
        index = rxn_index1.index(xx2)
        rxn_index2.append(index)
    else:
        print(f"{xx2} is not in the list.")
        exchange.remove(xx2)

In [ ]:
rxn_name_all=list(reac["Abbreviation"])
exchange_reaction_ids=[]
for i in rxn_index2:
    exchange_reaction_ids.append(rxn_name_all[int(i)])

In [ ]:
exchange2=[x+'e[e]' for x in exchange]
ex_rxn_mets_df=pd.DataFrame({'ID':exchange_reaction_ids,'Abbreviation':exchange2})
ex_rxn_mets_df = pd.merge(ex_rxn_mets_df,df3,on="Abbreviation")

In [ ]:
import cobra
import numpy as np
from scipy.stats import pearsonr

In [ ]:
model = cobra.io.read_sbml_model('iHil3966.xml')

In [ ]:
# Calculate the scenario where only one metabolite is restricted
# Assume exchange_reaction_ids, model, lower_bounds are already defined
lower_bounds = [1, 2, 3, 4, 10, 20, 30, 50, 100, 200, 300, 400, 500, 1000]
# Store scores for each metabolite

# Set standard lower bounds for all relevant compounds, establishing a baseline
standard_lower_bound = 0  # Assumed standard lower bound, set to 80% of maximum biomass, using this lower bound (10) the biomass is 3.55*0.8=2.84
# for exchange_id in exchange_reaction_ids:
#     model.reactions.get_by_id(exchange_id).lower_bound = standard_lower_bound

# Calculate the baseline growth rate
# baseline_solution = model.optimize()
# baseline_growth_rate = baseline_solution.objective_value

for exchange_id in exchange_reaction_ids:
    # Store growth rate increases and relative growth rates after each condition change
    growth_rate_increases = []
    growth_rates_relative_increase = []
    model.reactions.get_by_id(exchange_id).lower_bound = standard_lower_bound
    # Calculate the baseline growth rate
    baseline_solution = model.optimize()
    baseline_growth_rate = baseline_solution.objective_value
    for lb in lower_bounds:
        # Restore to baseline first
#         for reset_id in exchange_reaction_ids:
#             model.reactions.get_by_id(reset_id).lower_bound = standard_lower_bound
        
        # Change the lower bound of the current compound individually
        model.reactions.get_by_id(exchange_id).lower_bound = -lb
        
        # Perform FBA (Flux Balance Analysis)
        solution = model.optimize()
        
        # Record the growth rate under current conditions
        growth_rate = solution.objective_value
        
        # Calculate the increase in growth rate and the relative growth rate
        growth_rate_increase = growth_rate - baseline_growth_rate
        growth_rate_increases.append(growth_rate_increase)
        
        # If the baseline growth rate is 0 (to prevent division by zero error), set the relative growth rate to np.nan
        relative_increase = (growth_rate_increase / baseline_growth_rate * 100) if baseline_growth_rate else np.nan
        growth_rates_relative_increase.append(relative_increase)
    
    # Calculate the average growth rate increase, correlation score, and average relative growth rate for each compound
    average_growth_rate = np.mean(growth_rate_increases)
    average_relative_increase = np.mean(growth_rates_relative_increase)
    correlation, _ = pearsonr(lower_bounds, growth_rate_increases)
    
    # Store results
    scores[exchange_id] = {"average_growth_rate": average_growth_rate,
                           "average_relative_increase": average_relative_increase,
                           "correlation": correlation}

    model.reactions.get_by_id(exchange_id).lower_bound = -1000


In [ ]:
df = pd.DataFrame.from_dict(scores,orient='index')
df = df.sort_values(by="average_relative_increase", ascending=False)

In [ ]:
df_reset = df.reset_index()
df_reset.rename(columns={'index': 'Exchange Reaction ID'}, inplace=True)
df_results_final=pd.merge(ex_rxn_mets_df,df_reset)
df_results_final.sort_values(by="average_growth_rate", ascending=False).head(20)